In [1]:
# parameters
imgDim = [128, 128]
batch_size = 16
modelName = "mobileNetV2_normalized"

In [2]:
from keras.applications import mobilenet_v2
model = mobilenet_v2.MobileNetV2(weights=None, input_shape=[*imgDim, 2], classes=17)
print(model.summary())

][0]          
__________________________________________________________________________________________________
block_6_depthwise_relu (ReLU)   (None, 13, 13, 192)  0           block_6_depthwise_BN[0][0]       
__________________________________________________________________________________________________
block_6_project (Conv2D)        (None, 13, 13, 64)   12288       block_6_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_6_project_BN (BatchNormal (None, 13, 13, 64)   256         block_6_project[0][0]            
__________________________________________________________________________________________________
block_7_expand (Conv2D)         (None, 13, 13, 384)  24576       block_6_project_BN[0][0]         
__________________________________________________________________________________________________
block_7_expand_BN (BatchNormali (None, 13, 13, 384)  1536        block_7_expand[0][0]         

In [3]:
from keras import backend as K

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
from keras.optimizers import Adadelta
model.compile(loss='categorical_crossentropy', metrics=[f1], optimizer=Adadelta())

W1029 17:56:33.656908 15644 deprecation_wrapper.py:119] From C:\Users\tomni\Anaconda3\envs\celltracking\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [5]:
from dataGenerator import DataGenerator
from personal_data import location_data
dataGenerator_train = DataGenerator(location_data, batch_size=batch_size, inputDim=imgDim, normalize_data=True)
dataGenerator_val = DataGenerator(location_data, batch_size=batch_size, inputDim=imgDim, normalize_data=True, isTrainingData=False)

In [6]:
from keras import callbacks
import time
import os

# tensorboard --logdir ~/cytodata2019/tensorboard &
tensorboard = callbacks.TensorBoard(update_freq='batch',
    log_dir=os.path.join("tensorboard", modelName) + time.strftime("%Y%m%d-%H%M%S"))

In [7]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("inbetweenModel", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [8]:
from storingModel import save_model
for i in range(10):
    model.fit_generator(dataGenerator_train.generate(),
        validation_data=dataGenerator_val.generate(),
        callbacks=[tensorboard, checkpoint],
        steps_per_epoch=len(dataGenerator_train),
        validation_steps=len(dataGenerator_val),
        verbose=1, epochs=10)
    save_model(model, modelName + "_inbetween")

W1029 17:56:35.646972 15644 deprecation.py:323] From C:\Users\tomni\Anaconda3\envs\celltracking\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1029 17:56:49.502322 15644 deprecation_wrapper.py:119] From C:\Users\tomni\Anaconda3\envs\celltracking\lib\site-packages\keras\callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W1029 17:56:49.504317 15644 deprecation_wrapper.py:119] From C:\Users\tomni\Anaconda3\envs\celltracking\lib\site-packages\keras\callbacks.py:1125: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



In [0]:
from storingModel import save_model
save_model(model, modelName)